# Prepare the environment

In [ ]:
! pip install "xarray[complete]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

# Import the necessary libraries

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
from tqdm.notebook import tqdm, trange

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hres0_path = "gs://weatherbench2/datasets/hres_t0/2016-2022-6h-240x121_equiangular_with_poles_conservative.zarr"
era5_6h_150_path = "gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr"
era5_daily_150_path = "gs://weatherbench2/datasets/era5_daily/1959-2023_01_10-1h-240x121_equiangular_with_poles_conservative.zarr"

hres_path = "gs://weatherbench2/datasets/hres/2016-2022-0012-240x121_equiangular_with_poles_conservative.zarr"
graphcast_2018_path = "gs://weatherbench2/datasets/graphcast/2018/date_range_2017-11-16_2019-02-01_12_hours-240x121_equiangular_with_poles_conservative.zarr"
graphcast_2020_path = "gs://weatherbench2/datasets/graphcast/2020/date_range_2019-11-16_2021-02-01_12_hours-240x121_equiangular_with_poles_conservative.zarr"
graphcast_hres_init_2020_path = "gs://weatherbench2/datasets/graphcast_hres_init/2020/date_range_2019-11-16_2021-02-01_12_hours-240x121_equiangular_with_poles_conservative.zarr"
fuxi_path = "gs://weatherbench2/datasets/fuxi/2020-240x121_equiangular_with_poles_conservative.zarr"
ifs_ens_path = "gs://weatherbench2/datasets/ifs_ens/2018-2022-240x121_equiangular_with_poles_conservative.zarr"
ifs_ens_mean_path = "gs://weatherbench2/datasets/ifs_ens/2018-2022-240x121_equiangular_with_poles_conservative_mean.zarr"
pangu_path = "gs://weatherbench2/datasets/pangu/2018-2022_0012_240x121_equiangular_with_poles_conservative.zarr"
pangu_oper_path = "gs://weatherbench2/datasets/pangu_hres_init/2020_0012_240x121_equiangular_with_poles_conservative.zarr"
neuralgcm_path = "gs://weatherbench2/datasets/neuralgcm_deterministic/2020-240x121_equiangular_with_poles_conservative.zarr"
neuralgcm_ens_path = "gs://weatherbench2/datasets/neuralgcm_ens/2020-240x121_equiangular_with_poles_conservative.zarr"
neuralgcm_ens_mean_path = "gs://weatherbench2/datasets/neuralgcm_ens/2020-240x121_equiangular_with_poles_conservative_mean.zarr"
era5_forecast_path = "gs://weatherbench2/datasets/era5-forecasts/2020-240x121_equiangular_with_poles_conservative.zarr"
keisler_path = "gs://weatherbench2/datasets/keisler/2020-240x121_equiangular_with_poles_conservative.zarr"
sphericalcnn_path = "gs://weatherbench2/datasets/sphericalcnn/2020-240x121_equiangular_with_poles.zarr"

In [ ]:
work_dir = Path("/content/drive/MyDrive/WB2BinaryForecast/")
# work_dir = Path.cwd()

# Download the data to google drive

## Load the ERA5 and HRES t0 data

In [ ]:
era5_t2m6h = xr.open_zarr(era5_6h_150_path)["2m_temperature"].sel(time=pd.date_range(
    start=datetime.datetime(2019,12,31,0,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="6h")
           ).load()

In [ ]:
hres0_t2m6h = xr.open_zarr(hres0_path)["2m_temperature"].sel(time=pd.date_range(
    start=datetime.datetime(2019,12,31,0,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="6h")
           ).load()

In [ ]:
tp24h_era5 = xr.open_zarr(era5_6h_150_path)["total_precipitation_24hr"].sel(time=pd.date_range(
    start=datetime.datetime(2020,1,1,12,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="12h")
           ).load()

# Calaulate the 90th, 95th and 99th percentiles of the 24-hour accumulation of total precipitation (TP24h) and the 24-hour maximum of 2m temperature (T2M24h)

## T2M24h

### ERA5

In [ ]:
era5_t2m6h = xr.open_zarr(era5_6h_150_path)["2m_temperature"].sel(time=pd.date_range(
    start=datetime.datetime(2019,12,31,0,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="6h")
           ).load()

In [ ]:
tx24h_era5 = era5_t2m6h.rolling(time=4).max(dim="time").sel(time=pd.date_range(
    start=datetime.datetime(2020,1,1,12,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="12h"))
tx24h_era5

<ipython-input-31-a23df3ac7510>:1: DeprecationWarning: Reductions are applied along the rolling dimension(s) '['time']'. Passing the 'dim' kwarg to reduction operations has no effect.
  tx24h_era5 = era5_t2m6h.rolling(time=4).max(dim="time").sel(time=pd.date_range(


<xarray.DataArray '2m_temperature' (time: 732, longitude: 240, latitude: 121)> Size: 85MB
array([[[247.41321, 248.07083, 249.12105, ..., 251.11765, 249.28937,
         247.90855],
        [247.4405 , 248.09973, 249.12167, ..., 250.67055, 249.09453,
         247.84401],
        [247.46838, 248.13098, 249.127  , ..., 250.19406, 248.90172,
         247.78012],
        ...,
        [247.3393 , 248.00275, 249.25648, ..., 251.70721, 249.87363,
         248.0913 ],
        [247.36443, 248.02551, 249.21024, ..., 251.60347, 249.68132,
         248.03172],
        [247.38931, 248.04878, 249.1641 , ..., 251.38121, 249.48311,
         247.97076]],

       [[247.41321, 248.07083, 249.28519, ..., 247.01715, 246.83163,
         247.33662],
        [247.4405 , 248.09973, 249.32124, ..., 246.99127, 246.85225,
         247.30473],
        [247.46838, 248.13098, 249.36584, ..., 246.97127, 246.88072,
         247.26494],
...
        [243.87697, 245.40237, 247.40639, ..., 266.9786 , 265.36514,
         263.63467],
        [243.8711 , 245.29463, 247.15794, ..., 267.12204, 265.37616,
         263.64648],
        [243.86638, 245.19711, 246.89421, ..., 267.27994, 265.3802 ,
         263.65836]],

       [[244.51071, 244.84912, 244.53358, ..., 267.4282 , 265.60718,
         264.31027],
        [244.50598, 244.79453, 244.39215, ..., 267.48135, 265.59225,
         264.3123 ],
        [244.50067, 244.74115, 244.25304, ..., 267.50772, 265.57416,
         264.3154 ],
        ...,
        [244.52365, 245.00023, 244.98936, ..., 266.9786 , 265.66873,
         264.32794],
        [244.5192 , 244.94963, 244.83488, ..., 267.12204, 265.6445 ,
         264.32233],
        [244.51497, 244.89948, 244.68369, ..., 267.27994, 265.62143,
         264.3159 ]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * time       (time) datetime64[ns] 6kB 2020-01-01T12:00:00 ... 2021-01-01
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [ ]:
tx24h_era5_threshold = tx24h_era5.chunk(dict(time=-1)).quantile([0.9, 0.95, 0.99], dim="time").astype(np.float32).compute()
tx24h_era5_threshold

<xarray.DataArray '2m_temperature' (quantile: 3, longitude: 240, latitude: 121)> Size: 348kB
array([[[247.68526, 248.29765, 248.67233, ..., 274.1522 , 274.1399 ,
         274.132  ],
        [247.66515, 248.2334 , 248.55107, ..., 274.1477 , 274.13   ,
         274.13174],
        [247.64401, 248.1714 , 248.41768, ..., 274.15103, 274.13858,
         274.134  ],
        ...,
        [247.73213, 248.2719 , 248.99219, ..., 274.1563 , 274.1358 ,
         274.13434],
        [247.71767, 248.28526, 248.91304, ..., 274.14566, 274.13443,
         274.13275],
        [247.70108, 248.29909, 248.74388, ..., 274.15833, 274.13617,
         274.13226]],

       [[248.67123, 249.24373, 249.51765, ..., 274.45956, 274.3701 ,
         274.3454 ],
        [248.67427, 249.20164, 249.44522, ..., 274.46228, 274.36673,
         274.35007],
        [248.67767, 249.19165, 249.34016, ..., 274.45184, 274.37112,
         274.35385],
...
        [248.68895, 249.33846, 249.84314, ..., 274.42215, 274.37146,
         274.3532 ],
        [248.68079, 249.30586, 249.71951, ..., 274.4182 , 274.37457,
         274.34912],
        [248.67311, 249.27548, 249.61002, ..., 274.4372 , 274.37784,
         274.34637]],

       [[252.8345 , 253.34528, 252.81773, ..., 275.39917, 275.00497,
         274.8093 ],
        [252.80124, 253.219  , 252.6356 , ..., 275.37796, 275.00067,
         274.79935],
        [252.76753, 253.0929 , 252.46425, ..., 275.35687, 274.98813,
         274.78937],
        ...,
        [252.92493, 253.65062, 253.44733, ..., 275.42896, 275.00018,
         274.84326],
        [252.8953 , 253.55136, 253.21538, ..., 275.39792, 274.99707,
         274.8315 ],
        [252.86522, 253.45065, 253.01228, ..., 275.40588, 275.00314,
         274.82   ]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * quantile   (quantile) float64 24B 0.9 0.95 0.99

In [ ]:

tx24h_era5.to_netcdf(work_dir / "Data/Data_tx24h_150_2020_era5.nc")
tx24h_era5_threshold.to_netcdf(work_dir / "Threshold_tx24h_150_2020_era5.nc")

In [ ]:
era5_tx24h_extremes = xr.Dataset({#'anomality': (tx24h_era5-tx24h_era5_threshold).astype(np.float32),
                                  # "data": tx24h_era5,
                                  'events': tx24h_era5>tx24h_era5_threshold, })
era5_tx24h_extremes

<xarray.Dataset> Size: 64MB
Dimensions:    (latitude: 121, longitude: 240, time: 732, quantile: 3)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * time       (time) datetime64[ns] 6kB 2020-01-01T12:00:00 ... 2021-01-01
  * quantile   (quantile) float64 24B 0.9 0.95 0.99
Data variables:
    events     (time, longitude, latitude, quantile) bool 64MB False ... False

In [ ]:
era5_tx24h_extremes.to_netcdf(work_dir / "Extremes/Extremes_tx24h_150_2020_era5.nc")

### HRES t0

In [ ]:
hres0_t2m6h = xr.open_zarr(hres0_path)["2m_temperature"].sel(time=pd.date_range(
    start=datetime.datetime(2019,12,31,0,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="6h")
           ).load()

In [ ]:
tx24h_hres0 = hres0_t2m6h.rolling(time=4).max(dim="time").sel(time=pd.date_range(
    start=datetime.datetime(2020,1,1,12,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="12h"))
tx24h_hres0

<ipython-input-37-bf38f0d46466>:1: DeprecationWarning: Reductions are applied along the rolling dimension(s) '['time']'. Passing the 'dim' kwarg to reduction operations has no effect.
  tx24h_hres0 = hres0_t2m6h.rolling(time=4).max(dim="time").sel(time=pd.date_range(


<xarray.DataArray '2m_temperature' (time: 732, longitude: 240, latitude: 121)> Size: 85MB
array([[[245.58698, 246.79227, 247.81892, ..., 249.59995, 248.94101,
         247.97304],
        [245.615  , 246.79794, 247.82558, ..., 249.5874 , 248.88156,
         247.96455],
        [245.64165, 246.79393, 247.83707, ..., 249.70229, 248.8161 ,
         247.956  ],
        ...,
        [245.49255, 246.73402, 247.91362, ..., 249.81319, 249.04134,
         247.99988],
        [245.5243 , 246.75613, 247.85832, ..., 249.76791, 249.01662,
         247.99117],
        [245.55626, 246.7753 , 247.83131, ..., 249.67206, 248.9821 ,
         247.98204]],

       [[245.93057, 247.8812 , 248.58995, ..., 246.75836, 246.46861,
         246.34416],
        [245.92296, 247.86118, 248.59184, ..., 246.73581, 246.4562 ,
         246.33142],
        [245.93744, 247.8179 , 248.50424, ..., 246.74747, 246.43997,
         246.31517],
...
        [242.30965, 243.00352, 244.26021, ..., 266.40045, 264.9891 ,
         263.53198],
        [242.30313, 242.98836, 244.09027, ..., 266.5459 , 265.0711 ,
         263.55923],
        [242.2972 , 242.96457, 243.92366, ..., 266.67255, 265.1573 ,
         263.58456]],

       [[242.29227, 242.80682, 243.22826, ..., 266.94507, 265.45517,
         264.42328],
        [242.29218, 242.75008, 243.15341, ..., 266.9539 , 265.4599 ,
         264.43378],
        [242.29234, 242.6877 , 243.05977, ..., 266.94528, 265.4639 ,
         264.4449 ],
        ...,
        [242.30965, 242.9351 , 243.45793, ..., 266.71707, 265.37772,
         264.4025 ],
        [242.30313, 242.89694, 243.37424, ..., 266.8311 , 265.4097 ,
         264.40955],
        [242.2972 , 242.85373, 243.29689, ..., 266.90668, 265.43527,
         264.4163 ]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * time       (time) datetime64[ns] 6kB 2020-01-01T12:00:00 ... 2021-01-01
Attributes:
    long_name:      2 metre temperature
    short_name:     t2m
    standard_name:  unknown
    units:          K

In [ ]:
tx24h_hres0_threshold = tx24h_hres0.chunk(dict(time=-1)).quantile([0.9, 0.95, 0.99], dim="time").astype(np.float32).compute()
tx24h_hres0_threshold

<xarray.DataArray '2m_temperature' (quantile: 3, longitude: 240, latitude: 121)> Size: 348kB
array([[[245.69353, 246.22745, 247.11603, ..., 273.8495 , 273.85422,
         273.8883 ],
        [245.6961 , 246.20644, 247.00488, ..., 273.88144, 273.85187,
         273.88715],
        [245.70024, 246.22096, 246.83885, ..., 273.89682, 273.84546,
         273.88583],
        ...,
        [245.6514 , 246.34148, 247.42706, ..., 273.87683, 273.84744,
         273.88568],
        [245.66745, 246.3013 , 247.2875 , ..., 273.883  , 273.85233,
         273.88712],
        [245.6816 , 246.26448, 247.16345, ..., 273.87033, 273.8562 ,
         273.88806]],

       [[246.73157, 247.50279, 248.38934, ..., 274.08365, 274.0374 ,
         274.06125],
        [246.71529, 247.46568, 248.20569, ..., 274.0962 , 274.02295,
         274.06305],
        [246.69853, 247.42456, 248.16292, ..., 274.09406, 274.03238,
         274.06464],
...
        [246.77502, 247.60645, 248.51643, ..., 274.08044, 274.04315,
         274.06036],
        [246.76065, 247.57399, 248.41336, ..., 274.08496, 274.04257,
         274.05945],
        [246.74622, 247.53995, 248.48071, ..., 274.07956, 274.04205,
         274.05923]],

       [[251.30424, 251.89848, 251.41676, ..., 274.69675, 274.63654,
         274.53696],
        [251.27542, 251.78221, 251.49232, ..., 274.705  , 274.6289 ,
         274.5342 ],
        [251.24516, 251.66327, 251.60959, ..., 274.69498, 274.6222 ,
         274.53165],
        ...,
        [251.3722 , 252.19322, 252.04134, ..., 274.82822, 274.64374,
         274.5373 ],
        [251.349  , 252.1032 , 251.80928, ..., 274.76764, 274.6488 ,
         274.5367 ],
        [251.32692, 252.00381, 251.57776, ..., 274.71252, 274.6442 ,
         274.53708]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * quantile   (quantile) float64 24B 0.9 0.95 0.99

In [ ]:
tx24h_hres0.to_netcdf(work_dir / "Data/Data_tx24h_150_2020_hres0.nc")
tx24h_hres0_threshold.to_netcdf(work_dir / "Threshold_tx24h_150_2020_hres0.nc")

In [ ]:
hres0_tx24h_extremes = xr.Dataset({#'anomality': (tx24h_hres0-tx24h_hres0_threshold).astype(np.float32),
                                   #"data": tx24h_hres0,
                                'events': tx24h_hres0>tx24h_hres0_threshold, })
hres0_tx24h_extremes

<xarray.Dataset> Size: 64MB
Dimensions:    (latitude: 121, longitude: 240, time: 732, quantile: 3)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * time       (time) datetime64[ns] 6kB 2020-01-01T12:00:00 ... 2021-01-01
  * quantile   (quantile) float64 24B 0.9 0.95 0.99
Data variables:
    events     (time, longitude, latitude, quantile) bool 64MB False ... False

In [ ]:
hres0_tx24h_extremes.to_netcdf(work_dir / "Extremes/Extremes_tx24h_150_2020_hres0.nc")

## TP24h

In [ ]:
tp24h_era5 = xr.open_zarr(era5_6h_150_path)["total_precipitation_24hr"].sel(time=pd.date_range(
    start=datetime.datetime(2020,1,1,12,0,0),
    end=datetime.datetime(2021,1,1,1,0,0), freq="12h")
           ).load()

In [ ]:
tp24h_era5

<xarray.DataArray 'total_precipitation_24hr' (time: 732, longitude: 240,
                                              latitude: 121)> Size: 85MB
array([[[ 0.00000000e+00,  0.00000000e+00,  1.05283880e-05, ...,
          1.39031734e-04,  2.49485369e-04,  4.01417463e-04],
        [ 0.00000000e+00,  0.00000000e+00,  1.20544828e-05, ...,
          1.64696903e-04,  2.69224343e-04,  4.01953264e-04],
        [ 0.00000000e+00,  0.00000000e+00,  1.58044713e-05, ...,
          1.93472384e-04,  2.90131720e-04,  4.03483660e-04],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  5.82302255e-06, ...,
          1.12255628e-04,  1.91704472e-04,  3.96108022e-04],
        [ 0.00000000e+00,  0.00000000e+00,  7.16834620e-06, ...,
          1.22654310e-04,  2.13666281e-04,  3.97771451e-04],
        [ 0.00000000e+00,  0.00000000e+00,  8.49644584e-06, ...,
          1.29167835e-04,  2.32387480e-04,  3.99077428e-04]],

       [[ 4.33337118e-05,  4.37732451e-05,  3.86755964e-05, ...,
          5.72373665e-07,  1.15465509e-05,  1.56398673e-04],
        [ 4.41691773e-05,  4.85187229e-05,  4.48808423e-05, ...,
          2.62074792e-07,  1.31631168e-05,  1.62024051e-04],
        [ 4.48016726e-05,  5.36318294e-05,  5.46914307e-05, ...,
          5.65625783e-07,  1.57552731e-05,  1.69594234e-04],
...
          8.17911699e-04,  5.79873391e-04,  5.95783931e-04],
        [ 9.88020474e-06,  4.05685278e-05,  4.47845850e-05, ...,
          8.17048422e-04,  5.64747199e-04,  5.96532947e-04],
        [ 1.00702891e-05,  4.13789894e-05,  4.53157008e-05, ...,
          8.21596012e-04,  5.47988457e-04,  5.97212987e-04]],

       [[ 1.71231522e-06,  8.76084050e-06,  3.85978774e-06, ...,
          1.08518742e-03,  3.58190620e-04,  3.59362370e-04],
        [ 1.71231534e-06,  8.71328302e-06,  3.62422861e-06, ...,
          1.05462933e-03,  3.31485702e-04,  3.56704055e-04],
        [ 1.71231534e-06,  8.47477622e-06,  3.63585059e-06, ...,
          1.01638492e-03,  3.09223688e-04,  3.56089789e-04],
        ...,
        [ 1.71231545e-06,  8.41866313e-06,  3.30813532e-06, ...,
          1.13868865e-03,  4.57218295e-04,  3.67370638e-04],
        [ 1.71231534e-06,  8.59675583e-06,  3.52833763e-06, ...,
          1.11468346e-03,  4.26230923e-04,  3.64703708e-04],
        [ 1.71231545e-06,  8.74272519e-06,  3.46332172e-06, ...,
          1.10095856e-03,  3.92715214e-04,  3.61792103e-04]]],
      dtype=float32)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * time       (time) datetime64[ns] 6kB 2020-01-01T12:00:00 ... 2021-01-01
Attributes:
    long_name:   Total precipitation
    short_name:  tp
    units:       m

In [ ]:
tp24h_era5_threshold = tp24h_era5.chunk(dict(time=-1)).quantile([0.9, 0.95, 0.99], dim="time").astype(np.float32).compute()
tp24h_era5_threshold

<xarray.DataArray 'total_precipitation_24hr' (quantile: 3, longitude: 240,
                                              latitude: 121)> Size: 348kB
array([[[0.00060243, 0.00049152, 0.00046228, ..., 0.00246669,
         0.00211652, 0.00214297],
        [0.00060355, 0.00048468, 0.00043949, ..., 0.00247784,
         0.00212608, 0.00214201],
        [0.00060794, 0.00047818, 0.00043555, ..., 0.00250541,
         0.00215064, 0.00215042],
        ...,
        [0.00060244, 0.00049533, 0.00047552, ..., 0.00232936,
         0.00211971, 0.00210677],
        [0.00060031, 0.00049287, 0.00046409, ..., 0.00253471,
         0.0021189 , 0.00212209],
        [0.00059808, 0.0004882 , 0.00046639, ..., 0.0024574 ,
         0.00210505, 0.0021371 ]],

       [[0.00093246, 0.00082612, 0.00074355, ..., 0.00401032,
         0.00346261, 0.00310439],
        [0.000935  , 0.00081995, 0.00074117, ..., 0.0039946 ,
         0.00346552, 0.00310215],
        [0.0009389 , 0.0008149 , 0.00073687, ..., 0.00398485,
         0.00341189, 0.00309949],
...
        [0.00092895, 0.00084844, 0.00073586, ..., 0.00386903,
         0.00343278, 0.00314553],
        [0.00093075, 0.00084297, 0.00074543, ..., 0.00394343,
         0.00347176, 0.00311864],
        [0.00093178, 0.00083706, 0.00075784, ..., 0.00397285,
         0.00348347, 0.00310763]],

       [[0.00198994, 0.00193295, 0.00139853, ..., 0.00685882,
         0.00822698, 0.00767184],
        [0.0019997 , 0.00192544, 0.00136566, ..., 0.00701881,
         0.00827293, 0.00763758],
        [0.00201158, 0.00191704, 0.00133116, ..., 0.00706686,
         0.00828089, 0.00760092],
        ...,
        [0.00196915, 0.00194407, 0.00148535, ..., 0.00629509,
         0.00791044, 0.00776459],
        [0.00197541, 0.00193915, 0.00145616, ..., 0.00636168,
         0.00791521, 0.00773682],
        [0.00198221, 0.00193535, 0.00142645, ..., 0.00658411,
         0.00812976, 0.00770391]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * quantile   (quantile) float64 24B 0.9 0.95 0.99

In [ ]:
tp24h_era5.to_netcdf(work_dir / "Data/Data_tp24h_150_2020_era5.nc")
tp24h_era5_threshold.to_netcdf(work_dir / "Threshold_tp24h_150_2020_era5.nc")

In [ ]:
era5_tp24h_extremes = xr.Dataset({#'anomality': (tp24h_era5-tp24h_era5_threshold).astype(np.float32),
                                  #"data": tp24h_era5,
                                'events': tp24h_era5>tp24h_era5_threshold, })
era5_tp24h_extremes

<xarray.Dataset> Size: 64MB
Dimensions:    (latitude: 121, longitude: 240, time: 732, quantile: 3)
Coordinates:
  * latitude   (latitude) float64 968B -90.0 -88.5 -87.0 ... 87.0 88.5 90.0
  * longitude  (longitude) float64 2kB 0.0 1.5 3.0 4.5 ... 355.5 357.0 358.5
  * time       (time) datetime64[ns] 6kB 2020-01-01T12:00:00 ... 2021-01-01
  * quantile   (quantile) float64 24B 0.9 0.95 0.99
Data variables:
    events     (time, longitude, latitude, quantile) bool 64MB False ... False

In [ ]:
era5_tp24h_extremes.to_netcdf(work_dir / "Extremes/Extremes_tp24h_150_2020_era5.nc")

## Calculate more thresholds

In [ ]:
quantile_list = [0.8, 0.85, 0.9, 0.93, 0.95, 0.96, 0.97, 0.98, 0.99]


In [ ]:
tx24h_era5 = xr.open_dataset(work_dir / "Data/Data_tx24h_150_2020_era5.nc")["2m_temperature"].load()
tx24h_hres0 = xr.open_dataset(work_dir / "Data/Data_tx24h_150_2020_hres0.nc")["2m_temperature"].load()
tp24h_era5 = xr.open_dataset(work_dir / "Data/Data_tp24h_150_2020_era5.nc")["total_precipitation_24hr"].load()

In [ ]:
tx24h_era5_threshold = tx24h_era5.chunk(dict(time=-1)).quantile(quantile_list, dim="time").astype(np.float32)
tx24h_era5_threshold.to_netcdf(work_dir / "Threshold_tx24h_150_2020_era5.nc")

tx24h_hres0_threshold = tx24h_hres0.chunk(dict(time=-1)).quantile(quantile_list, dim="time").astype(np.float32)
tx24h_hres0_threshold.to_netcdf(work_dir / "Threshold_tx24h_150_2020_hres0.nc")

tp24h_era5_threshold = tp24h_era5.chunk(dict(time=-1)).quantile(quantile_list, dim="time").astype(np.float32)
tp24h_era5_threshold.to_netcdf(work_dir / "Threshold_tp24h_150_2020_era5.nc")

In [ ]:
era5_tx24h_extremes = xr.Dataset({'events': tx24h_era5>tx24h_era5_threshold, })
era5_tx24h_extremes.to_netcdf(work_dir / "Extremes/Extremes_tx24h_150_2020_era5.nc")

hres0_tx24h_extremes = xr.Dataset({'events': tx24h_hres0>tx24h_hres0_threshold, })
hres0_tx24h_extremes.to_netcdf(work_dir / "Extremes/Extremes_tx24h_150_2020_hres0.nc")

era5_tp24h_extremes = xr.Dataset({'events': tp24h_era5>tp24h_era5_threshold, })
era5_tp24h_extremes.to_netcdf(work_dir / "Extremes/Extremes_tp24h_150_2020_era5.nc")

# Load the thresholds

In [ ]:
tx24h_era5_threshold = xr.open_dataset(work_dir / "Threshold_tx24h_150_2020_era5.nc")["2m_temperature"].load()
tx24h_hres0_threshold = xr.open_dataset(work_dir / "Threshold_tx24h_150_2020_hres0.nc")["2m_temperature"].load()
tp24h_era5_threshold = xr.open_dataset(work_dir / "Threshold_tp24h_150_2020_era5.nc")["total_precipitation_24hr"].load()

# Convert the continuous forecasts to binary forecasts

## T2M24h

### From deterministic forecasts

In [ ]:
def calculate_t2max_extremes(model_name, resolution, year, lead_time, recalc=False):
    save_extremes_path = work_dir / "Extremes/Extremes_tx24h_{}_{}_{}_{}d.nc".format(
        str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))
    save_data_path = work_dir / "Data/Data_tx24h_{}_{}_{}_{}d.nc".format(
        str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))

    if save_extremes_path.exists() and (not recalc): # 判断是否已经完成计算，或者需要重新计算
        print("Already saved:", save_extremes_path)
    else:
        if save_data_path.exists():
            forecast_data = xr.open_dataset(save_data_path)["2m_temperature"]
        else:
            forecast_data = xr.open_zarr(forecast_path_dict[model_name])["2m_temperature"]

            time_range = pd.date_range(datetime.datetime(2020,1,1,0,0,0), datetime.datetime(2021,1,1,1,0,0), freq="12h")
            prediction_timedelta = np.intersect1d(np.arange((lead_time-1)*24+6, lead_time*24+6, 6).astype('timedelta64[h]'),
                                                  forecast_data.prediction_timedelta.data.astype('timedelta64[h]'))
            # print(forecast_data.prediction_timedelta.data.astype('timedelta64[h]'))

            forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data),
                                              prediction_timedelta=prediction_timedelta).max(dim="prediction_timedelta").compute()

            forecast_data["time"] = forecast_data["time"] + np.timedelta64(lead_time, 'D')
            forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data))

            if "lat" in forecast_data.coords:
                forecast_data = forecast_data.rename({'lat': 'latitude', 'lon': 'longitude'})

            forecast_data.to_netcdf(save_data_path)

        if model_name in ["hres", "hres_ens_mean", "hres_ens"]:
            t2max_threshlod = tx24h_hres0_threshold
        else:
            t2max_threshlod = tx24h_era5_threshold


        forecast_extremes = xr.Dataset({#'anomality': (forecast_data-t2max_threshlod).astype(np.float32),
                                        #'data': forecast_data,
                                  'events': forecast_data>t2max_threshlod, })
        forecast_extremes.to_netcdf(save_extremes_path)
        print("Save file:", save_extremes_path)


In [ ]:
initial_times = [0, 12]
lead_times = np.arange(1, 11)
forecast_path_dict = {"graphcast": graphcast_2020_path, "graphcast-oper":graphcast_hres_init_2020_path,
                      "pangu": pangu_path, "pangu-oper": pangu_oper_path,
                      "hres": hres_path,"hres_ens_mean": ifs_ens_mean_path, "hres_ens": ifs_ens_path,
                      "era5-forecast": era5_forecast_path,
                      # "keisler": keisler_path, "sphericalcnn":sphericalcnn_path,
                      "fuxi":fuxi_path,
                      # "neuralgcm": neuralgcm_path
                      }

In [ ]:
resolution = 1.50
year = 2020

for model_name in tqdm(forecast_path_dict.keys()):

  for lead_time in tqdm(lead_times, leave=False):
    calculate_t2max_extremes(model_name, resolution, year, lead_time)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

### From ensemble forecasts

In [ ]:
def calculate_t2max_extremes(model_name, resolution, year, lead_time, recalc=False):
    for number in trange(1, 51, leave=False):
        save_extremes_path = work_dir / "Extremes/Extremes_tx24h_{}_{}_{}_{}d_{}em.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2), str(number).zfill(2))
        save_data_path = work_dir / "Data/Data_tx24h_{}_{}_{}_{}d_{}em.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2), str(number).zfill(2))

        if save_extremes_path.exists() and save_data_path.exists() and (not recalc): # 判断是否已经完成计算，或者需要重新计算
            print("Already saved:", save_extremes_path)
        else:
            if save_data_path.exists():
                forecast_data = xr.open_dataset(save_data_path)["2m_temperature"]
            else:
                forecast_data = xr.open_zarr(forecast_path_dict[model_name])["2m_temperature"]

                time_range = pd.date_range(datetime.datetime(2020,1,1,0,0,0), datetime.datetime(2021,1,1,1,0,0), freq="12h")
                prediction_timedelta = np.intersect1d(np.arange((lead_time-1)*24+6, lead_time*24+6, 6).astype('timedelta64[h]'),
                                                      forecast_data.prediction_timedelta.data.astype('timedelta64[h]'))
                # print(forecast_data.prediction_timedelta.data.astype('timedelta64[h]'))

                forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data), number=number,
                                                  prediction_timedelta=prediction_timedelta).max(dim="prediction_timedelta").compute()

                forecast_data["time"] = forecast_data["time"] + np.timedelta64(lead_time, 'D')
                forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data))

                if "lat" in forecast_data.coords:
                    forecast_data = forecast_data.rename({'lat': 'latitude', 'lon': 'longitude'})

                forecast_data.to_netcdf(save_data_path)

            if model_name in ["hres", "hres_ens_mean", "hres_ens"]:
                t2max_threshlod = tx24h_hres0_threshold
            else:
                t2max_threshlod = tx24h_era5_threshold


            forecast_extremes = xr.Dataset({#'anomality': (forecast_data-t2max_threshlod).astype(np.float32),
                                            #'data': forecast_data,
                                      'events': forecast_data>t2max_threshlod, })
            forecast_extremes.to_netcdf(save_extremes_path)
            print("Save file:", save_extremes_path)


In [ ]:
initial_times = [0, 12]
lead_times = np.arange(9, 10)
forecast_path_dict = {"graphcast": graphcast_2020_path, "graphcast-oper":graphcast_hres_init_2020_path,
                      "pangu": pangu_path, "pangu-oper": pangu_oper_path,
                      "hres": hres_path,"hres_ens_mean": ifs_ens_mean_path, "hres_ens": ifs_ens_path,
                      "era5-forecast": era5_forecast_path,
                      # "keisler": keisler_path, "sphericalcnn":sphericalcnn_path,
                      "fuxi":fuxi_path,
                      # "neuralgcm": neuralgcm_path
                      }

In [ ]:
resolution = 1.50
year = 2020

# for model_name in tqdm(forecast_path_dict.keys()):
for model_name in tqdm(["hres_ens"]):

  for lead_time in tqdm(lead_times, leave=False):
    calculate_t2max_extremes(model_name, resolution, year, lead_time)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_01em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_02em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_03em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_04em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_05em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_06em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_07em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_08em.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tx24h_150_2020_hres_ens_09d_09em.nc
Save file: /content/drive/MyDrive/WB2BinaryFor

#### Concat the 50 ensemble members to one dataset

In [ ]:
%%time
resolution = 1.50
lead_times = np.arange(1, 11)
year = 2020
model_name = "hres_ens"

for lead_time in tqdm(lead_times, leave=False):
    ens_extremes = []
    for number in trange(1, 51, leave=False):
        save_number_extremes_path = work_dir / "Extremes/Extremes_tx24h_{}_{}_{}_{}d_{}em.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2), str(number).zfill(2))
        if not save_number_extremes_path.exists():
            print(save_number_extremes_path)
        
        test_data = xr.open_dataset(save_number_extremes_path)["events"]
        ens_extremes.append(test_data)
    ens_extremes = xr.concat(ens_extremes, dim="number")
    ens_extremes["number"] = np.arange(1, 51)
    ens_extremes = ens_extremes.sum(dim="number") # /50
    save_extremes_path = work_dir / "Extremes/Extremes_tx24h_{}_{}_{}_{}d.nc".format(
        str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))
    ens_extremes.astype("i1").to_netcdf(save_extremes_path)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 1min 12s, sys: 2min 57s, total: 4min 9s
Wall time: 14min 31s


# TP24h

## From deterministic forecasts

In [ ]:
def calculate_tp24h_extremes(model_name, resolution, year, lead_time, recalc=False):
    save_extremes_path = work_dir / "Extremes/Extremes_tp24h_{}_{}_{}_{}d.nc".format(
        str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))
    save_data_path = work_dir / "Data/Data_tp24h_{}_{}_{}_{}d.nc".format(
        str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))

    if save_extremes_path.exists() and (not recalc):
        print("Already saved:", save_extremes_path)
    else:
        if save_data_path.exists():
            forecast_data = xr.open_dataset(save_data_path)["total_precipitation_24hr"]
        else:
            forecast_data = xr.open_zarr(forecast_path_dict[model_name])
            if "total_precipitation_24hr_from_6hr" in forecast_data.data_vars:
                forecast_data = forecast_data.rename({"total_precipitation_24hr_from_6hr": "total_precipitation_24hr"})
                forecast_data["total_precipitation_24hr"] = forecast_data["total_precipitation_24hr"]/1000 # fuxi的total_precipitation_24hr单位为mm
            forecast_data = forecast_data["total_precipitation_24hr"]

            time_range = pd.date_range(datetime.datetime(2020,1,1,0,0,0), datetime.datetime(2021,1,1,1,0,0), freq="12h")
            prediction_timedelta = np.timedelta64(lead_time, 'D')
            # print(forecast_data.prediction_timedelta.data.astype('timedelta64[h]'))

            forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data),
                                              prediction_timedelta=prediction_timedelta)

            forecast_data["time"] = forecast_data["time"] + np.timedelta64(lead_time, 'D')
            forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data))

            if "lat" in forecast_data.coords:
                forecast_data = forecast_data.rename({'lat': 'latitude', 'lon': 'longitude'})

            forecast_data.to_netcdf(save_data_path)

        tp24h_threshlod = tp24h_era5_threshold
        forecast_extremes = xr.Dataset({#'anomality': (forecast_data-t2max_threshlod).astype(np.float32),
                                        #'data': forecast_data,
                                  'events': forecast_data>tp24h_threshlod, })
        forecast_extremes.to_netcdf(save_extremes_path)
        print("Save file:", save_extremes_path)


In [ ]:
initial_times = [0, 12]
lead_times = np.arange(1, 11)
forecast_path_dict = {"graphcast": graphcast_2020_path, "graphcast-oper":graphcast_hres_init_2020_path,
                      "pangu": pangu_path, "pangu-oper": pangu_oper_path,
                      "hres": hres_path,"hres_ens_mean": ifs_ens_mean_path,
                      "era5-forecast": era5_forecast_path,
                      # "keisler": keisler_path, "sphericalcnn":sphericalcnn_path,
                      "fuxi":fuxi_path,
                      # "neuralgcm": neuralgcm_path
                      }

In [ ]:
resolution = 1.50
year = 2020

for model_name in tqdm(["hres", "hres_ens_mean", "graphcast", "graphcast-oper", "fuxi"]):
  for lead_time in tqdm(lead_times, leave=False):
    calculate_tp24h_extremes(model_name, resolution, year, lead_time)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_01d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_02d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_03d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_04d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_05d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_06d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_07d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_08d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_09d.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hr

  0%|          | 0/10 [00:00<?, ?it/s]

Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_01d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_02d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_03d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_04d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_05d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_06d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_07d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_08d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_mean_09d.nc
Save file: /content/drive/MyDrive/WB2Binar

...

Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_01d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_02d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_03d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_04d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_05d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_06d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_07d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_08d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast_09d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_20

  0%|          | 0/10 [00:00<?, ?it/s]

Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_01d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_02d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_03d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_04d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_05d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_06d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_07d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_08d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_graphcast-oper_09d.nc
Save file: /content/drive/MyDrive/WB2

  0%|          | 0/10 [00:00<?, ?it/s]

Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_01d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_02d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_03d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_04d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_05d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_06d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_07d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_08d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_09d.nc
Save file: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_fuxi_10d.nc


## From ensemble forecasts

In [ ]:
def calculate_tp24h_extremes(model_name, resolution, year, lead_time, recalc=False):
    for number in trange(1, 51, leave=False):

        save_extremes_path = work_dir / "Extremes/Extremes_tp24h_{}_{}_{}_{}d_{}em.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2), str(number).zfill(2))
        save_data_path = work_dir / "Data/Data_tp24h_{}_{}_{}_{}d_{}em.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2), str(number).zfill(2))

        if save_extremes_path.exists() and save_data_path.exists() and (not recalc): # 判断是否已经完成计算，或者需要重新计算
            print("Already saved:", save_extremes_path)
        else:
            if save_data_path.exists():
                forecast_data = xr.open_dataset(save_data_path)["total_precipitation_24hr"]
            else:
                forecast_data = xr.open_zarr(forecast_path_dict[model_name])
                if "total_precipitation_24hr_from_6hr" in forecast_data.data_vars:
                    forecast_data = forecast_data.rename({"total_precipitation_24hr_from_6hr": "total_precipitation_24hr"})
                    forecast_data["total_precipitation_24hr"] = forecast_data["total_precipitation_24hr"]/1000 # fuxi的total_precipitation_24hr单位为mm
                forecast_data = forecast_data["total_precipitation_24hr"]

                time_range = pd.date_range(datetime.datetime(2020,1,1,0,0,0), datetime.datetime(2021,1,1,1,0,0), freq="12h")
                prediction_timedelta = np.timedelta64(lead_time, 'D')
                # print(forecast_data.prediction_timedelta.data.astype('timedelta64[h]'))

                forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data), number=number,
                                                  prediction_timedelta=prediction_timedelta)

                forecast_data["time"] = forecast_data["time"] + np.timedelta64(lead_time, 'D')
                forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data))

                if "lat" in forecast_data.coords:
                    forecast_data = forecast_data.rename({'lat': 'latitude', 'lon': 'longitude'})

                forecast_data.to_netcdf(save_data_path)

            tp24h_threshlod = tp24h_era5_threshold
            forecast_extremes = xr.Dataset({#'anomality': (forecast_data-t2max_threshlod).astype(np.float32),
                                            #'data': forecast_data,
                                      'events': forecast_data>tp24h_threshlod, })
            forecast_extremes.to_netcdf(save_extremes_path)
            print("Save file:", save_extremes_path)


In [ ]:
initial_times = [0, 12]
lead_times = np.arange(7, 8)
forecast_path_dict = {"graphcast": graphcast_2020_path, "graphcast-oper":graphcast_hres_init_2020_path,
                      "pangu": pangu_path, "pangu-oper": pangu_oper_path,
                      "hres": hres_path,"hres_ens_mean": ifs_ens_mean_path, "hres_ens": ifs_ens_path,
                      "era5-forecast": era5_forecast_path,
                      # "keisler": keisler_path, "sphericalcnn":sphericalcnn_path,
                      "fuxi":fuxi_path,
                      # "neuralgcm": neuralgcm_path
                      }

In [ ]:
resolution = 1.50
year = 2020

# for model_name in tqdm(forecast_path_dict.keys()):
for model_name in tqdm(["hres_ens"]):

  for lead_time in tqdm(lead_times, leave=False):
    calculate_tp24h_extremes(model_name, resolution, year, lead_time)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_26em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_27em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_28em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_29em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_30em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_31em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_32em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_33em.nc
Already saved: /content/drive/MyDrive/WB2BinaryForecast/Extremes/Extremes_tp24h_150_2020_hres_ens_07d_34em.nc
Already sa

### Concat the 50 ensemble members to one dataset

In [ ]:
%%time
resolution = 1.50
lead_times = np.arange(1, 11)
year = 2020
model_name = "hres_ens"

for lead_time in tqdm(lead_times, leave=False):
    ens_extremes = []
    for number in trange(1, 51, leave=False):
        save_number_extremes_path = work_dir / "Extremes/Extremes_tp24h_{}_{}_{}_{}d_{}em.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2), str(number).zfill(2))
        if not save_number_extremes_path.exists():
            print(save_number_extremes_path)
        
        test_data = xr.open_dataset(save_number_extremes_path)#["events"]
        ens_extremes.append(test_data)
    ens_extremes = xr.concat(ens_extremes, dim="number")
    ens_extremes["number"] = np.arange(1, 51)
    ens_extremes = ens_extremes.sum(dim="number")
    save_extremes_path = work_dir / "Extremes/Extremes_tp24h_{}_{}_{}_{}d.nc".format(
        str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))
    ens_extremes.astype("i1").to_netcdf(save_extremes_path)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 1min 17s, sys: 2min 31s, total: 3min 49s
Wall time: 15min 50s


# Concat the raw forecasts to form a single netcdf file

In [ ]:
resolution = 1.50
year = 2020
initial_times = [0, 12]
lead_times = np.arange(1, 11)
model_list_heatwave = ["hres", "hres_ens_mean", "era5-forecast",
               "pangu-oper","graphcast-oper", "pangu", "graphcast","fuxi"]
model_list_rainfall = ["hres", "hres_ens_mean", "graphcast", "graphcast-oper", "fuxi"]

forecast_path_dict = {"graphcast": graphcast_2020_path, "graphcast-oper":graphcast_hres_init_2020_path,
                      "pangu": pangu_path, "pangu-oper": pangu_oper_path,
                      "hres": hres_path,"hres_ens_mean": ifs_ens_mean_path,
                      "era5-forecast": era5_forecast_path,
                      # "keisler": keisler_path, "sphericalcnn":sphericalcnn_path,
                      "fuxi":fuxi_path,
                      # "neuralgcm": neuralgcm_path
                      }

time_range = pd.date_range(datetime.datetime(2020,1,1,0,0,0), datetime.datetime(2021,1,1,1,0,0), freq="24h")

In [ ]:
models_data = []
for model_name in tqdm(model_list_heatwave):
    lead_time_data = []
    for lead_time in tqdm(lead_times, leave=False):
        forecast_path = work_dir / "Data/Data_tx24h_{}_{}_{}_{}d.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))

        forecast_data = xr.open_dataset(forecast_path)["2m_temperature"]
        forecast_data["time"] = forecast_data["time"] - np.timedelta64(1, 'D')
        forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data))
        lead_time_data.append(forecast_data)

    lead_time_data = xr.concat(lead_time_data, dim="lead_time")
    lead_time_data["lead_time"] = lead_times

    models_data.append(xr.Dataset({model_name: lead_time_data}))
models_data = xr.merge(models_data)
models_data.to_netcdf(work_dir / "Forecasts_tx24h_150_2020.nc")

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
models_data = []

for model_name in tqdm(model_list_rainfall):
    lead_time_data = []
    for lead_time in tqdm(lead_times, leave=False):
        forecast_path = work_dir / "Data/Data_tp24h_{}_{}_{}_{}d.nc".format(
            str(resolution).replace(".", "").ljust(3, "0"), str(year), model_name, str(lead_time).zfill(2))

        forecast_data = xr.open_dataset(forecast_path)["total_precipitation_24hr"]
        forecast_data["time"] = forecast_data["time"] - np.timedelta64(1, 'D')
        forecast_data = forecast_data.sel(time=np.intersect1d(time_range, forecast_data.time.data))

        lead_time_data.append(forecast_data)

    lead_time_data = xr.concat(lead_time_data, dim="lead_time")
    lead_time_data["lead_time"] = lead_times

    models_data.append(xr.Dataset({model_name: lead_time_data}))
models_data = xr.merge(models_data)
models_data.to_netcdf(work_dir / "Forecasts_tp24h_150_2020.nc")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]